In [ ]:
import torch
import torchvision
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

In [ ]:
example = torch.rand(1, 3, 224, 224)

In [ ]:
torch.onnx.export(vgg16, example, "vgg16.onnx", verbose=True)

In [ ]:
! head vgg16.onnx

In [ ]:
! pip install onnx
! pip install tensorflow
! pip install tensorflow-addons
! pip install onnx_tf 

In [1]:
import onnx
from onnx_tf.backend import prepare

In [2]:
model = onnx.load('vgg16.onnx')
tf_rep = prepare(model) 

2022-02-26 14:49:23.572426: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
! wget https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg

--2022-02-26 14:49:38--  https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg
Распознаётся upload.wikimedia.org (upload.wikimedia.org)… 91.198.174.208
Подключение к upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 156145 (152K) [image/jpeg]
Сохранение в: «1200px-Cat03.jpg.1»

1200px-Cat03.jpg.1  100%[===================>] 152,49K  --.-KB/s    за 0,1s    

2022-02-26 14:49:39 (1,08 MB/s) - «1200px-Cat03.jpg.1» сохранён [156145/156145]



In [4]:
import torchvision.transforms as transforms

transform_pipeline = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])


def prepare(img_path):
    image = Image.open(img_path)
    img_data = transform_pipeline(image).unsqueeze(0)
    data = np.array(img_data)
    return data

In [5]:
import numpy as np
from IPython.display import display
from PIL import Image

img = prepare('1200px-Cat03.jpg')

In [7]:
output = tf_rep.run(np.asarray(img, dtype=np.float32))

In [8]:
np.argmax(output)

285

In [9]:
tf_rep.export_graph('vgg16.pb')

2022-02-26 14:50:34.156599: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: vgg16.pb/assets


INFO:tensorflow:Assets written to: vgg16.pb/assets


In [ ]:
! ls

In [10]:
! ls vgg16.pb

assets         saved_model.pb variables


In [11]:
import tensorflow as tf

In [12]:
modeltf = tf.saved_model.load("vgg16.pb")

In [13]:
modeltf

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x171f55280>

In [14]:
infer = modeltf.signatures["serving_default"]

In [15]:
tf.constant(img)

<tf.Tensor: shape=(1, 3, 224, 224), dtype=float32, numpy=
array([[[[1.5981677 , 1.6324171 , 1.6837914 , ..., 1.8379141 ,
          1.8550389 , 1.8550389 ],
         [1.5981677 , 1.6324171 , 1.6837914 , ..., 1.8721637 ,
          1.8892884 , 1.8892884 ],
         [1.5981677 , 1.6324171 , 1.6837914 , ..., 1.9064132 ,
          1.9064132 , 1.8892884 ],
         ...,
         [1.1015497 , 1.1357993 , 1.0844251 , ..., 1.6324171 ,
          1.6324171 , 1.6152923 ],
         [1.1186745 , 1.1529241 , 1.1015497 , ..., 1.6837914 ,
          1.6837914 , 1.6837914 ],
         [1.0501755 , 1.0844251 , 1.0844251 , ..., 1.7180408 ,
          1.7180408 , 1.7351656 ]],

        [[1.6232493 , 1.6757703 , 1.7457983 , ..., 1.8858544 ,
          1.8683473 , 1.8508403 ],
         [1.6057423 , 1.6757703 , 1.7457983 , ..., 1.8858544 ,
          1.8858544 , 1.8508403 ],
         [1.6407562 , 1.7107843 , 1.7633053 , ..., 1.8858544 ,
          1.8508403 , 1.7983193 ],
         ...,
         [0.71288526, 0.765406

In [16]:
output = infer(tf.constant(img))

In [17]:
values = next(iter(output.values()))

In [18]:
np.argmax(values)

285